## Covariance-Matrix Adaptation Evolution Strategy (CMA-ES)

Cet algorithme rentre dans la catégorie des algorithmes à stratégie d'évolution. Il est très performant sur les modèles continus, bruités, mal conditionnés, etc. En revanche, si l'espace de recherche est à peu de dimensions, ou si on connaît le gradient de notre fonction, d'autres méthodes sont plus efficaces. (voir *optimisation non-linéaire* avec les méthodes à base de descente de gradient, `scipy.optimize` sous Python)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import animation

import numpy as np
import scipy.interpolate as sp

def anim_to_html(anim):
    plt.close(anim._fig)
    return anim.to_html5_video()

animation.Animation._repr_html_ = anim_to_html

Pour comprendre le principe de ces algorithmes, on va partir d'une fonction assez chaotique que l'on génère à partir de points tirés au hasard :

In [ ]:
x = np.linspace(0, 10, 30)
y = np.random.uniform(0, -10, 30)
tck = sp.splrep(x, y)

x_ = np.linspace(0, 10, 200)

# this function returns the value of each position
# we show it in the Y axis.
# the lower, the better.
def get_value(x):
    return sp.splev(x, tck)

y_ = get_value(x_)
plt.plot(x_, y_)

Ici on manipule une fonction à base de splines. Pour trouver le minimum de cette fonction, on peut commencer par tirer $\lambda$ points uniformément répartis et les évaluer, puis garder les $\mu$ meilleurs :

In [ ]:
λ, μ = 100, 20

plt.xlim((0, 10))
plt.ylim((-12, 0))
plt.plot(x_, y_)

x = np.random.uniform(0, 10, λ)
y = sp.splev(x, tck)
best = sorted(x, key=lambda x: sp.splev(x, tck))[:μ]
plt.plot(x, y, 'o', color='#aaaaaa')
plt.plot(best, sp.splev(best, tck), 'o', color='#aa3a3a')

À partir des $\mu$ meilleurs points, de leur moyenne et de leur écart-type, on peut créer une nouvelle loi normale, et tirer à nouveau $\lambda$ nouveaux points.

In [ ]:
fig = plt.figure(figsize=(7, 5))
gs = gridspec.GridSpec(5, 1)

top = plt.subplot(gs[0])
ax = plt.subplot(gs[1:-1], sharex=top)
bottom = plt.subplot(gs[-1], sharex=top)
top.set_yticks([])
bottom.set_yticks([])
    
λ, μ = 100, 20
best = np.random.uniform(0, 10, λ)
all_ = []

ax.plot(x_, y_)

def init():
    global best
    best = np.random.uniform(0, 10, λ)
    return all_
    
def update(frame):
    global best, all_
    [a.remove() for a in all_]
    all_.clear()
    
    # we model 'potentially good' solutions 
    # as a normal distribution
    # using the μ best solutions found last time
    avg, std = np.mean(best), np.std(best)
    
    # we simulate λ new solutions based on
    # this normal distribution
    current = np.random.normal(avg, std, λ)
    current = current[(current < 10) & (current > 0)]
    
    # we choose the new μ best solutions from the recently 
    # simulated solutions
    best = sorted(current, key=get_value)[:μ]
    
    all_ += ax.plot(current, sp.splev(current, tck), 'o', color="#aaaaaa", alpha=.5)
    all_ += ax.plot(best, sp.splev(best, tck), 'o', color="#aa3a3a")

    count, bins, ignored = bottom.hist(current, color="#aaaaaa", alpha=.5, bins=30)
    all_ += ignored
    all_ += top.plot(bins, 1/(std * np.sqrt(2 * np.pi)) *
                     np.exp( - (bins - avg)**2 / (2 * std**2) ),
                     linewidth=2, color='#aaaaaa')
    
    count, bins, ignored = bottom.hist(best, color="#aa3a3a", bins=30)
    all_ += ignored
    avg, std = np.mean(best), np.std(best)
    all_ += top.plot(bins, 1/(std * np.sqrt(2 * np.pi)) *
                     np.exp( - (bins - avg)**2 / (2 * std**2) ),
                     linewidth=2, color='#aa3a3a')
    
    top.relim()
    top.autoscale_view()
    
    return all_

animation.FuncAnimation(fig, update, init_func=init, frames=20, interval=500, blit=True)


On peut généraliser cette méthode à $n$ dimensions. Illustrons avec les cartes 2D que nous générions pour le recuit simulé: 

In [ ]:
n_boulders = 150
boulders = np.random.uniform(0., 1., (n_boulders, 3))
boulders[:, 2] = 1/(3 ** np.random.choice(range(2, 4), (n_boulders)))

def eval_boulders(x, y, boulders = boulders):
    res = 0
    for i in range(boulders.shape[0]):
        eval_ = (x - boulders[i, 0]) ** 2 + (y - boulders[i, 1]) ** 2 - boulders[i, 2] ** 2
        if eval_ < 0:
            res += eval_
    return res


X, Y = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
Z = np.zeros(X.shape)

for i in range(n_boulders):
    eval_ = (X - boulders[i, 0]) ** 2 + (Y - boulders[i, 1]) ** 2 - boulders[i, 2] ** 2
    Z[eval_ < 0] += eval_[eval_<0]

fig = plt.figure(figsize=(7, 7))
plt.contour(X, Y, Z, alpha=.5)

On utilise maintenant une loi normale multivariée qui implique des matrices de covariance. La diagonale de la matrice de covariance est liée à la variance d'une distribution dans chacune des dimensions, et les termes extra-diagonaux expriment la corrélation entre chacune des dimensions. Des termes extra-diagonaux positifs représentent une « ellipse » penchée d'un côté et négatifs de l'autre.

En diagonalisant la matrice de covariance, on retrouve les directions dans lesquelles se répartissent nos échantillons.  
C'est d'ailleurs sur cette diagonalisation que repose l'[Analyse en Composantes Principales](https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales).

In [ ]:
from matplotlib.patches import Ellipse
import scipy.linalg as sl

covs = [np.asarray([[2, 0], [0, 1]]),
        np.asarray([[2, 1], [1, 2]]),
        np.asarray([[2, -1], [-1, 2]]),]

fig, ax = plt.subplots(1, len(covs), figsize=(15, 5))

for i, cov in enumerate(covs):
    
    x = np.random.multivariate_normal([0, 0], cov, 100)
    e, v = sl.eig(cov)
    
    angle = 0 if i == 0 else np.degrees(np.arccos(np.clip(np.dot(v[:,0], [0, 1]), -1, 1)))
    
    ax[i].plot(*x.T, 'o')
    ax[i].add_patch(Ellipse([0, 0], 2*np.sqrt(5.991)*e[0].real, 2*np.sqrt(5.991)*e[1].real,
                            angle,
                            fc="#aaaaaa"))
    
    ax[i].set_axis_off()
    ax[i].axis('square')
    ax[i].set_xlim([-6, 6])
    ax[i].set_ylim([-6, 6])
    ax[i].set_title("{}\n{}".format(*cov.tolist()))


In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.gca()
    
λ, μ = 100, 20
best = np.random.uniform(0., 1., (λ, 2))
all_ = []
avgs = []

ax.contour(X, Y, Z, alpha=.5)

def init():
    global best
    best = np.random.uniform(0., 1., (λ, 2))
    return all_
    
def update(frame):
    global best, all_, avgs
    [a.remove() for a in all_]
    all_.clear()
    
    avg, cov = np.mean(best, axis=0), np.cov(best.T)
    avgs.append(avg)
    current = np.random.multivariate_normal(avg, cov, λ)
    mask = np.bitwise_and(np.bitwise_and(current[:,0] > 0, current[:, 1]>0),
                          np.bitwise_and(current[:, 0] < 1, current[:, 1] < 1))

    current = current[mask]
    best = sorted(current, key=lambda x: eval_boulders(*x))[:μ]
    best = np.array(best)

    all_ += ax.plot(*current.T, 'o', color='#aaaaaa')
    all_ += ax.plot(*np.array(best).T, 'o', color='#aa3a3a')

    eigval, eigvec = sl.eig(cov)
    for i, v in enumerate(eigval):
        all_ += ax.plot(*np.c_[avg - 2*np.sqrt(5.991)*v.real*eigvec[:, i],
                               avg + 2*np.sqrt(5.991)*v.real*eigvec[:, i]], '-k', lw=2)
    
    return all_

animation.FuncAnimation(fig, update, init_func=init, frames=15, interval=500, blit=True)


La convergence est similaire à celle de l'exemple précédent en une dimension. Il est intéressant de voir l'évolution de la matrice de covariance, représentée ici par les directions/dimensions de ses vecteurs propres.

### Mais voilà...

En pratique, ça ne peut pas fonctionner comme ça... Souvenez-vous que:
1. ce qui fonctionne en dimension 2 ne fonctionne pas nécessairement en dimension supérieure (malédiction de la dimension);
2. on peut être piégé dans de mauvais minima locaux à cause du bruit;
3. les métaphores fleurissent pour comparer stratégie d'évolution (fitness function, generation, matrice de covariance) et descente du gradient (objective function, iteration, matrice Hessienne); et la descente de gradient à pas constant ne fonctionne pas dans le cas général...


Du coup, on va plutôt écrire qu'on recherche une population sous la forme :
$$x_i \sim m + \sigma \mathcal{N_i}(0, C)$$
 
avec :

- $m$, la moyenne de l'échantillon précédent (disons pour le moment, les $\mu$ meilleurs parents),
- $C$, une matrice de covariance,
- $\sigma$, un pas d'itération.

Intuitivement, $m$ situe le voisinage de la meilleure solution trouvée jusqu'ici, $C$ donne la forme de l'ellipsoïde, et $\sigma$ est un pas d'itération (similaire au pas de la descente de gradient).

La mise à jour de $m$ est presque directe, celle de $\sigma$ et de C va être un peu plus complexe...

### Mise à jour de $m$, stratégies d'évolution

- Stratégie élitiste $(\mu + \lambda)-$ES : sélection parmi les parents et les nouveaux échantillons;
- Stratégie non-élitiste $(\mu, \lambda)$-ES: sélection parmi les nouveaux échantillons seulement (exemple ci-dessus).

En CMA-ES, la stratégie classique est $({\mu}/{\mu_w}, \lambda)-$ES, c'est à dire avec **sélection parmi les nouveaux échantillons et pondération des échantillons en fonction de leur score**: on trie les échantillons $x_i$ par ordre décroissant de score, et on fait un produit scalaire avec un vecteur $w$ de poids décroissants:

$m \leftarrow \sum_{i=1}^{\mu}w_i\cdot x_i = m + \sigma y_w = m + \sigma\sum_{i=1}^\mu w_i \cdot y_i$ avec $\sum w_i = 1$ et $\mu_w = \dfrac{1}{\sum w_i^2} \sim \dfrac{\lambda}{4}$

Cette recombinaison pondérée est équivalente à un mécanisme de sélection.

<div class="alert alert-warning">
**Note importante**: Nous allons voir beaucoup de paramètres défiler, mais la bonne nouvelle est que les paramètres par défaut fonctionnent a priori bien quel que soit le problème. En pratique, le seul argument du CMA-ES que vous pourriez avoir à régler est la *taille de la population*.
</div>

### Mise à jour de $\sigma$

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.gca()

it = 4

ax.contour(X, Y, Z, alpha=.5)

ax.plot(*np.asarray(avgs).T, '-k', lw=2)

ax.arrow(avgs[0][0], avgs[0][1],
         avgs[it][0] - avgs[0][0], avgs[it][1] - avgs[0][1],
         fc='r', ec='r', lw=2, length_includes_head=True,
         head_width=.01)

ax.axis('equal')


Pour la mise à jour de $\sigma$, l'intuition repose sur le rapport entre la longueur du chemin parcouru par la moyenne de l'échantillon à chaque itération (en noir) et le chemin direct de la moyenne initiale vers la moyenne à l'échantillon courant (en rouge). Si le chemin noir « tourne en rond », alors il faut réduire $\sigma$; au contraire, si le chemin noir parcourt l'équivalent du chemin rouge mais en plus d'itérations, il faut augmenter $\sigma$.

On initialise $p_\sigma = 0$, $c_\sigma \sim 4/n$ et $d_\sigma \sim 1$. Intuitivement, $p_\sigma$ représente l'évolution du trait noir.

$$p_\sigma \leftarrow (1 - c_\sigma) p_\sigma + \sqrt{1-(1-c_\sigma)^2} \sqrt{\mu_w} \cdot y_w$$

Note : Le terme en $\sqrt{1-(1-c_\sigma)^2}$ est un facteur de normalisation.

$$\sigma \leftarrow \sigma \cdot e^{\dfrac{c_\sigma}{d_\sigma}\left(\dfrac{||p_\sigma||}{E ||\mathcal{N}(0, I)||}-1\right)}$$

Si le terme dans l'exponentielle est négatif, $\sigma$ diminue, s'il est positif $\sigma$ augmente.

### Mise à jour de la matrice de covariance (CMA)

La matrice de covariance va être adaptée suivant deux axes (en plus d'une inertie suivant son historique) :

- l'évolution du chemin parcouru (*rank-one update*) : la matrice de covariance va être déformée dans le sens de parcours de $m$ ;

$$p_C \leftarrow (1 - c_C) p_C + \sqrt{1-(1-c_C)^2} \sqrt{\mu_w} \cdot y_w$$
$$C \leftarrow (1 - c_{cov})\, C + c_{cov}\, p_c\,p_c^T$$

- l'évolution de la matrice de covariance relative à la distribution des $\mu$ meilleurs points (*rank-$\mu$ update*).

$$C_\mu = \sum_{i=1}^{\mu} w_i\,y_i\, y_i^T$$
$$C \leftarrow (1 - c_{cov})\, C + c_{cov}\,C_\mu$$


avec $c_{cov} \sim \dfrac{\mu_w}{n^2}$

En pratique, on combine en pondérant ces deux stratégies avec deux coefficients.

### L'algorithme complet sur le problème précédent

On utilise un module Python clé en main pour cet algorithme.

A priori la population (le paramètre $\lambda$) peut être laissé à sa valeur par défaut (qui dépend de la dimension de l'espace de solution). Pour l'animation courante, le paramètre par défaut est 6, mais il est un peu grossi pour une animation visuellement un peu plus parlante.

On n'a pas accès à la matrice de covariance et au pas utilisé par l'algorithme à chaque itération, alors la croix tracée est reformée à partir des points tirés par l'algorithme...

In [ ]:
import cma

fig = plt.figure(figsize=(7, 7))
ax = fig.gca()

ax.contour(X, Y, Z, alpha=.5)
all_ = []

ax.set_xlim([0, 1])
ax.set_ylim([0, 1])


def cma_exec():

    es = cma.CMAEvolutionStrategy(2 * [0.5], .5, {'popsize': 15})

    while not es.stop():
        x = np.asarray(es.ask())
        yield x
        es.tell(x, [eval_boulders(*x_) for x_ in x])
        
def update(x):
    global all_
    [a.remove() for a in all_]
    all_.clear()
    
    avg, cov = np.mean(x, axis=0), np.cov(x.T)
    all_ += ax.plot(*x.T, 'o', color='#aa3a3a')
    
    eigval, eigvec = sl.eig(cov)
    for i, v in enumerate(eigval):
        all_ += ax.plot(*np.c_[avg - 2*np.sqrt(5.991)*v.real*eigvec[:, i],
                               avg + 2*np.sqrt(5.991)*v.real*eigvec[:, i]], '-k', lw=2)
        
    return all_

animation.FuncAnimation(fig, update, frames=cma_exec, interval=500, blit=True)


## Problème à optimiser

On cherche à placer les villes européennes suivantes sur une carte en ne connaissant que les distances les séparant.

In [ ]:
from stochastic.data import cities, distances

n = len(cities)
print(
    "{} cities:\n  {}".format(
        n, "\n  ".join(
            [" ".join([format(el, '<10')
                       for el in cities[8*i:8*(i+1)]])
             for i in range(4)]
        )
    )
)

La matrice des distances suivante est fournie.  
**Note:** Elle a été calculée à partir des vraies coordonnées lat/lon de ces villes puis normalisée, mais l'idée est de donner des coordonnées x-y euclidiennes à ces villes.

In [ ]:
distances

<div class="alert alert-warning">
    <b>Questions :</b>

<ul>
<li>Modéliser le problème par une fonction à optimiser puis le résoudre avec l'algorithme CMA-ES.</li>
<li>Afficher sur une carte la position des villes.</li>
<li>La méthode `cma.fmin` prend en argument optionnel le gradient de la fonction à optimiser pour guider plus précisément la recherche. Comparer les nombres d'itérations avec sans gradient</li>
<li>Y aurait-il une méthode plus efficace que le CMA-ES pour résoudre ce problème?</li>
</ul>
</div>

- Il est recommandé d'utiliser la méthode `cma.fmin()` (voir l'aide) plutôt que la méthode manuelle ci-dessus (qui nous a servi à afficher la convergence).
- La méthode `plt.annotate` permettra d'afficher le nom de la ville sur un plot à côté du point associé à ses coordonnées.

In [ ]:
# %load solutions/03-cities.py


In [ ]:
solution = cma.fmin(func, x0.ravel(), 1)

In [ ]:
solution = cma.fmin(func, x0.ravel(), 1, gradf=func_der)

In [ ]:
import scipy.optimize as sopt

solution = sopt.fmin_bfgs(func, x0.ravel(), fprime=func_der, retall=True)

## Post-processing

Comme toutes les rotations et symétries (miroir) de notre carte sont des solutions équivalentes de notre problème, on va remettre la carte dans le bon sens:
- Rome et Copenhague sont à la même longitude (ou presque)
- À partir de deux villes dont on sait qu'elles sont à l'Est/Ouest l'une de l'autre, on décide si un miroir est nécessaire.


In [ ]:
# %load solutions/03-postprocess.py


In [ ]:
# %load solutions/03-plot_cities.py
